In [0]:
import dlt
from pyspark.sql.functions import col

# Configurações de conexão com o PostgreSQL
jdbc_url = "jdbc:postgresql://dpg-cuciu352ng1s73b52bgg-a.ohio-postgres.render.com:5432/transactions_yz0m"
jdbc_properties = {
    "user": "transactions_yz0m_user",
    "password": "66BsNmXaNh3btK1ZY8GLbnrFmwUFK9xY",
    "driver": "org.postgresql.Driver"
}

# Nome da tabela no PostgreSQL
postgres_table = "transactions"

# Carregar apenas os dados incrementais do PostgreSQL
@dlt.view(
    comment="Carrega os dados incrementais de transações do PostgreSQL"
)
def raw_transactions_incremental():
    # Obter o último `transaction_date` da tabela existente
    last_transaction_date = (
        spark.sql("SELECT MAX(transaction_date) AS last_transaction_date FROM raw_transactions")
        .collect()[0]["last_transaction_date"]
    )

    # Criar consulta incremental
    if last_transaction_date:
        print(f"Última transação registrada: {last_transaction_date}")
        query = f"""
            (SELECT * FROM {postgres_table}
             WHERE transaction_date > '{last_transaction_date}') AS t
        """
    else:
        print("Nenhum dado encontrado na tabela. Carregando todos os dados.")
        query = f"(SELECT * FROM {postgres_table}) AS t"

    # Carregar dados do PostgreSQL
    return (
        spark.read.format("jdbc")
        .option("url", jdbc_url)
        .option("dbtable", query)
        .option("user", jdbc_properties["user"])
        .option("password", jdbc_properties["password"])
        .option("driver", jdbc_properties["driver"])
        .load()
    )

# Atualizar ou inserir dados na tabela existente
@dlt.table(
    name="raw_transactions",
    comment="Tabela Delta gerenciada com dados incrementais de transações"
)
def raw_transactions():
    # Adicionar os dados incrementais à tabela existente
    return dlt.read("raw_transactions_incremental")
